In [6]:
pic_='F'
tracking_uri_="http://34.128.104.38:5000"

In [7]:
# Parameters
pic_ = "F"
tracking_uri_ = "http://34.128.104.38:5000"


In [8]:
import pandas as pd 
import numpy as np
import os
import scipy
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")


from sklearn.model_selection import train_test_split
from catboost import Pool, EShapCalcType, EFeaturesSelectionAlgorithm,CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score,cohen_kappa_score,roc_auc_score,log_loss
import os

from google.cloud import storage
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'config/gcp_store.json'
client = storage.Client()

from multiprocessing import Pool as pool_x

from itertools import repeat
import mlflow
from mlflow.tracking import MlflowClient


In [9]:
def parallel_FE_catboost(i):
    global pic_,status_, train_x, valid_x, train_y, valid_y, columns_data, strategy_
    train_pool=Pool(data=train_x,label=train_y)
    valid_pool=Pool(data=valid_x,label=valid_y)
    
    experiment_name = "Feature Selection"
    ## check if the experiment already exists
    if not mlflow.get_experiment_by_name(experiment_name):
        mlflow.create_experiment(name=experiment_name) 
    experiment = mlflow.get_experiment_by_name(experiment_name)
    ### check finished experiment
    client = MlflowClient()
    exp_r = client.get_experiment_by_name("Feature Selection")
    client_run=mlflow.search_runs([exp_r.experiment_id], order_by=["metrics.f1_micro DESC"])
    if ((client_run.shape[0]>0)&("params.condition" in client_run.columns)&
        ("params.algorithm" in client_run.columns)&("params.strategy" in client_run.columns)):
        client_run=client_run.loc[(client_run["params.condition"]==status_)&
                                  (client_run["params.algorithm"]=='catboost')&
                                  (client_run["params.strategy"]==strategy_)]
        if str(i) in client_run['params.num_feature'].to_list():
            print('Experiment with number of feature:',i, ' already done. It will be skipped.')
            return ''
    mlflow.set_tracking_uri(tracking_uri_)
    with mlflow.start_run(experiment_id = experiment.experiment_id,
                          run_name=status_+'-'+datetime.now().strftime("%d/%m/%Y %H:%M:%S")):
        model=CatBoostClassifier(iterations=100, random_seed=1234)
        summary = model.select_features(
                train_pool,
                eval_set=valid_pool,
                features_for_select=columns_data,
                num_features_to_select=i,
                steps=1,
                algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
                shap_calc_type=EShapCalcType.Regular,
                train_final_model=True,
                logging_level='Silent',
            )
        pred_labels=model.predict(valid_pool).tolist()
        preds_proba_=model.predict_proba(valid_pool)
        
        
        f1_micro=f1_score(valid_y, pred_labels,average='micro')
        log_loss_=log_loss(valid_y,preds_proba_)
        
        mlflow.log_param("num_feature", i)
        mlflow.log_param('feature_name',summary['selected_features_names'])
        mlflow.log_param('condition',status_)
        mlflow.log_param('algorithm',"catboost")        
        mlflow.log_param('strategy',strategy_)

        mlflow.log_metric("f1_micro", f1_micro)
        mlflow.log_metric("log_loss", log_loss_)
        
    return ''

In [10]:
def run_FE_catboost():
    y1=range(5,6)
    with pool_x(28) as p:
        p.map(parallel_FE_catboost,y1)
    

### Without Artificial Data

In [11]:
print('RUN DATA WITHOUT ARTIFICIAL DATA\n\n')
print('===========RUN Data w Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +".csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='no artificial'
run_FE_catboost()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='no artificial'
run_FE_catboost()
print('===========FINISH=======================')
print('\n\nFINISH')

RUN DATA WITHOUT ARTIFICIAL DATA


===========RUN Data w Outlier===========


Process ForkPoolWorker-26:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-11:
Process ForkPoolWorker-17:
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-3:
Process ForkPoolWorker-21:
Process ForkPoolWorker-14:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Process ForkPoolWorker-10:
Process ForkPoolWorker-8:
Process ForkPoolWorker-22:
Process ForkPoolWorker-7:
Process ForkPoolWorker-9:
Process ForkPoolWorker-25:
Process ForkPoolWorker-18:
Process ForkPoolWorker-1:
Process ForkPoolWorker-13:
Process ForkPoolWorker-23:
Process ForkPoolWorker-19:
Process ForkPoolWorker-2:
Process ForkPoolWorker-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-28:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-20:
Process ForkPoolWorker-24:
Traceback (most recent call last):
Traceback (most recent call last):
Tracebac

  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, 

KeyboardInterrupt
  File "/opt/tljh/user/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/opt/tljh/user/lib/python3.8/multiprocessing/queues.py", line 355, in get
    with self._rlock:
  File "/opt/tljh/user/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/opt/tljh/user/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/tmp/ipykernel_6002/1823895406.py", line 24, in parallel_FE_catboost
    with mlflow.start_run(experiment_id = experiment.experiment_id,
KeyboardInterrupt
  File "/opt/tljh/user/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/tljh/user/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._

KeyboardInterrupt: 

### ENN-BORDER SMOTE

In [ ]:
print('RUN DATA WITH ENN-BORDER SMOTE\n\n')
print('===========RUN Data w Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_enn_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='enn_border_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_enn_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='enn_border_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('\n\nFINISH')

### ENN-SMOTE

In [ ]:
print('RUN DATA WITH ENN-SMOTE\n\n')
print('===========RUN Data w Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_enn_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='enn_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_enn_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='enn_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('\n\nFINISH')

### TL-SMOTE

In [ ]:
print('RUN DATA WITH TL-SMOTE\n\n')
print('===========RUN Data w Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_tl_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='tl_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_tl_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='tl_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('\n\nFINISH')

### TL-BORDER SMOTE

In [ ]:
print('RUN DATA WITH TL-SMOTE\n\n')
print('===========RUN Data w Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_tl_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='w_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='tl_border_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('===========RUN Data w/o Outlier===========')
data_sample=pd.read_csv("gs://bps-gcp-bucket/MLST2023/preprocessing/sample_"+str(pic_) +"_no_outlier_tl_border_smote.csv",sep=',')
data_sample['nama_valid']=data_sample.nama_valid.apply(lambda y: str(y)[:6])
data_sample=data_sample.sort_values('nama_valid')
status_='wo_outlier_'
columns_data=['B1_p15', 'B2_p15', 'B3_p15',
       'B4_p15', 'B5_p15', 'B6_p15', 'B7_p15', 'B8_p15', 'B8A_p15', 'B11_p15',
       'B12_p15', 'NDVI_p50', 'NDWI_p50', 'NDBI_p50', 'SAVI_p50', 'EVI_p50',
       'GNDVI_p50']
X=data_sample[columns_data]
y=data_sample['nama_valid']
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.2,stratify=y)
strategy_='tl_border_smote'
run_FE_catboost()
print('===========FINISH=======================')
print('\n\nFINISH')